In [21]:
import tensorflow as tf
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2690884698727621462,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2048061004348675525
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4985044352
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14849858936765229874
 physical_device_desc: "device: 0, name: GeForce GTX 1660 SUPER, pci bus id: 0000:06:00.0, compute capability: 7.5",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 73088402124301748
 physical_device_desc: "device: XLA_GPU device"]

In [22]:
tf.__version__

'2.2.0'

In [2]:
tf.debugging.set_log_device_placement(True)

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint
import os
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from numpy import expand_dims
from keras import models
from keras import layers
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.python.keras.layers import Dense, Dropout, Input
from tensorflow.python.keras.models import Model
from tensorflow.keras.layers import Input, Dense

model = models.Sequential()
#1층
model.add(Conv2D(input_shape=(224,224,3), filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#2층
model.add(Conv2D(input_shape=(224,224,64),filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#Max Pooling
model.add(MaxPooling2D(pool_size =(2,2), strides=(2,2)))
#3층
model.add(Conv2D(input_shape=(112,112,64),filters=128, kernel_size=(3,3), strides=(1,1), padding ='same', activation='relu'))
#4층
model.add(Conv2D(input_shape=(112,112,128),filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#Max Pooling
model.add(MaxPooling2D(pool_size =(2,2), strides=(2,2)))
#5층
model.add(Conv2D(input_shape=(56,56,128), filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#6층
model.add(Conv2D(input_shape=(56,56,256), filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#7층
model.add(Conv2D(input_shape=(56,56,256), filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
#8층
model.add(Conv2D(input_shape=(28,28,256), filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#9층
model.add(Conv2D(input_shape=(28,28,512), filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#10층
model.add(Conv2D(input_shape=(28,28,512), filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
#11층
model.add(Conv2D(input_shape=(14,14,512), filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#12층
model.add(Conv2D(input_shape=(14,14,512), filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#13층
model.add(Conv2D(input_shape=(14,14,512), filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
#14층 (fc1)
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
#15층 (fc2)
model.add(Dense(4096,activation='relu'))
model.add(Dense(2048,activation='relu'))
model.add(Dense(1024,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/tas

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       2

In [5]:
###vgg16으로 custom data training해보기###
# https://aihub.or.kr/aidata/33
# 위해물품 엑스레이 이미지 dataset

In [6]:
# dataset이 용량이 너무 커서 34개의 카테고리 중 1개인, 술에 관해예측해보기
# 4096에서 34로 바로 연결하지않고, 2048,1024,512개의 unit을 거치는 Fully Connected Layer로 연결
# feature 정보의 손실을 줄이기 위함

In [7]:
## Labelimg를 통해 labeling을 했고, 이를 csv로 변환한후, tfrecord로 다시 변환한다.

In [38]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob('C:/Users/user/Desktop/github/study/2.논문 읽고 구현해보기/02.VGGnet(2014)/Alcohol/Alcohol/Single_Default' + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    for directory in ['train','test']:
        image_path = os.path.join(os.getcwd(), 'images/{}'.format(directory))
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv('data/{}_labels.csv'.format(directory), index=None)
        print('Successfully converted xml to csv.')


main()

Successfully converted xml to csv.
Successfully converted xml to csv.


In [8]:
#labelimg로 xray상의 alcohol에 라벨링
#train = 125개 ,test = 335개
train_dir = './Alcohol/Alcohol/vgg16_custom/train'
test_dir = './Alcohol/Alcohol/vgg16_custom/test'

IM_WIDTH = 220
IM_HEIGHT = 220

In [9]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=180, # 회전 쵀대 180도
      width_shift_range=0.2, # 좌우 이동 최대 이미지 가로사이즈 20%
      height_shift_range=0.2, # 상하 이동 최대 이미지  세로사이즈 20%
      horizontal_flip=True, # 좌우 반전 실행
      vertical_flip=True # 상하 반전 실행 
      )

In [10]:
#train_datagen.flow_from_directory() : 디렉토리에서 이미지를 읽을 객체를 생성
#train_datagen.flow_from_directory(
#        이미지 경로,       : TRAIN_DIR (/dental_image/train/)
#        target_size=(이미지가로, 이미지세로), IM_WIDTH (220) x IM_HEIGHT(220)
#        batch_size=한번에 리턴할 이미지의 개수, 16개
#        class_mode=분류방법, categorical (다중분류) binary(0 1 로 이중 분류)
#        shuffle=이미지를 섞을 여부
#)
        
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=16,
        class_mode='categorical',
        shuffle=True)

Found 109 images belonging to 1 classes.


In [11]:
# 검증에 사용될 이미지를 읽을 객체
#테스트에서는 이미지를 변형 할 필요가 없으므로
#ImageDataGenerator에 값을 주지 않음
#이미지가 변형되지 않고 같은 이미지 리턴
test_datagen = ImageDataGenerator( 
     rescale=1./255  
)
#test_datagen.flow_from_directory() : 디렉토리에서 이미지를 읽을 객체를 생성
#test_datagen.flow_from_directory(
#        이미지 경로,       : TEST_DIR (/dental_image/test/)
#        target_size=(이미지가로, 이미지세로), IM_WIDTH (220) x IM_HEIGHT(220)
#        batch_size=한번에 리턴할 이미지의 개수, 10개
#        class_mode=분류방법, categorical (다중분류) binary(0 1 로 이중 분류)
#        shuffle=이미지를 섞을 여부 False 섞지 않음
#)
        
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(IM_WIDTH, IM_HEIGHT),
        batch_size=16,
        class_mode='categorical',
        shuffle=False)

Found 16 images belonging to 1 classes.


In [12]:
train_generator.n

109

In [13]:
test_generator.n

16

In [14]:
print(train_generator.batch_size)
print(test_generator.batch_size)

16
16


In [15]:
img, label = train_generator.next()
img

array([[[[0.9805573 , 0.98108995, 0.98345095],
         [0.9815829 , 0.982959  , 0.98304456],
         [0.9824556 , 0.9805432 , 0.9841802 ],
         ...,
         [0.9803922 , 0.9843138 , 0.9843138 ],
         [0.9803922 , 0.9843138 , 0.9843138 ],
         [0.9653491 , 0.9689037 , 0.96927065]],

        [[0.98327345, 0.9807629 , 0.984013  ],
         [0.98072106, 0.98398495, 0.98170334],
         [0.9841646 , 0.98067915, 0.9822748 ],
         ...,
         [0.9811264 , 0.9835796 , 0.9835796 ],
         [0.980555  , 0.984151  , 0.984151  ],
         [0.9803922 , 0.9843138 , 0.9843138 ]],

        [[0.9833716 , 0.98372006, 0.98098594],
         [0.98129255, 0.98341346, 0.98098195],
         [0.9838925 , 0.98125064, 0.98195213],
         ...,
         [0.9843138 , 0.9810846 , 0.9810846 ],
         [0.9843138 , 0.9805131 , 0.9805131 ],
         [0.9838631 , 0.9808428 , 0.9808428 ]],

        ...,

        [[0.9803922 , 0.98206687, 0.98206687],
         [0.98259723, 0.9803922 , 0.98259723]

In [16]:
img.shape

(16, 220, 220, 3)

In [17]:
label

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [18]:
model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics=['acc'])

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0


In [20]:
model.fit(train_generator, epochs=20)

Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Epoch 1/20


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at <ipython-input-19-d4d9553b7342>:1) ]] [Op:__inference_train_function_2231]

Function call stack:
train_function
